In [19]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [2]:
file_path='../raw_data/fake_posts.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.head()

,Title,Content
0,Let's Talk about the Importance of Foot Care\n,"Hey there health enthusiasts! Today, let's cha..."
1,Random Health Benefits of Practicing Tai Chi\n\n,"Hey guys, it's your favorite health and lifest..."
2,Get Your Heart Pumping with These Seasonal Fit...,"Hey there, fitness enthusiasts! Today, I'm her..."
3,The Power of Herbal Teas for Optimal Health\n,"Hey health enthusiast friends! Today, I want t..."
4,The Ins and Outs of Proper Handwashing\n,"Yo yo yo, what up my peeps? Today I wanna talk..."


In [5]:
df['ID'] = range(1, len(df) + 1)
df.set_index('ID', inplace=True)

In [6]:
df.head()

,Title,Content
ID,,
1,Let's Talk about the Importance of Foot Care\n,"Hey there health enthusiasts! Today, let's cha..."
2,Random Health Benefits of Practicing Tai Chi\n\n,"Hey guys, it's your favorite health and lifest..."
3,Get Your Heart Pumping with These Seasonal Fit...,"Hey there, fitness enthusiasts! Today, I'm her..."
4,The Power of Herbal Teas for Optimal Health\n,"Hey health enthusiast friends! Today, I want t..."
5,The Ins and Outs of Proper Handwashing\n,"Yo yo yo, what up my peeps? Today I wanna talk..."


In [7]:
df['Title']=df['Title'].str.strip()

In [10]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [8]:
from sentence_transformers import SentenceTransformer

# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')


/Users/srividhyaleeladharan/.pyenv/versions/3.10.6/envs/project_4y/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/srividhyaleeladharan/.pyenv/versions/3.10.6/envs/project_4y/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:

# Split text into sentences
df['sentences'] = df['Content'].str.split('.')

In [17]:
# Get sentence embeddings
df['embeddings'] = df['sentences'].apply(lambda x: model.encode(x).tolist())

In [16]:
df.head()

,Title,Content,sentences,embeddings
ID,,,,
1,Let's Talk about the Importance of Foot Care,"Hey there health enthusiasts! Today, let's cha...","[Hey there health enthusiasts! Today, let's ch...","[[-0.10931964963674545, 0.10040649771690369, -..."
2,Random Health Benefits of Practicing Tai Chi,"Hey guys, it's your favorite health and lifest...","[Hey guys, it's your favorite health and lifes...","[[-0.10931964963674545, 0.10040649771690369, -..."
3,Get Your Heart Pumping with These Seasonal Fit...,"Hey there, fitness enthusiasts! Today, I'm her...","[Hey there, fitness enthusiasts! Today, I'm he...","[[-0.10931964963674545, 0.10040649771690369, -..."
4,The Power of Herbal Teas for Optimal Health,"Hey health enthusiast friends! Today, I want t...","[Hey health enthusiast friends! Today, I want ...","[[-0.10931964963674545, 0.10040649771690369, -..."
5,The Ins and Outs of Proper Handwashing,"Yo yo yo, what up my peeps? Today I wanna talk...","[Yo yo yo, what up my peeps? Today I wanna tal...","[[-0.06615692377090454, 0.012751547619700432, ..."


In [21]:
len(df)

999

In [24]:
def extract_tags(sentences):
    # Skip if there are not enough sentences for clustering
    if len(sentences) < 3:
        return []

    # Get sentence embeddings
    embeddings = model.encode(sentences)

    # Number of clusters (tags) you want to extract
    num_clusters = 3

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)

    # Get cluster centers
    cluster_centers = kmeans.cluster_centers_

    # Find the closest sentence in each cluster to be the tag
    tags = []
    for center in cluster_centers:
        distances = np.linalg.norm(embeddings - center, axis=1)
        tag_index = np.argmin(distances)
        tags.append(sentences[tag_index])

    return tags

In [27]:
df['sentences'] = df['Content'].str.split('. ')

In [28]:
df['tags'] = df['sentences'].apply(extract_tags)

/Users/srividhyaleeladharan/.pyenv/versions/3.10.6/envs/project_4y/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/srividhyaleeladharan/.pyenv/versions/3.10.6/envs/project_4y/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/srividhyaleeladharan/.pyenv/versions/3.10.6/envs/project_4y/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/srividhya

In [29]:
df.head()

,Title,Content,sentences,embeddings,tags
ID,,,,,
1,Let's Talk about the Importance of Foot Care,"Hey there health enthusiasts! Today, let's cha...","[He, ther, healt, enthusiasts, Today, let', ch...","[[-0.05392976105213165, -0.008646907284855843,...","[health, a, o]"
2,Random Health Benefits of Practicing Tai Chi,"Hey guys, it's your favorite health and lifest...","[He, guys, it', you, favorit, healt, an, lifes...","[[0.05749974772334099, -0.010059642605483532, ...","[body, , a]"
3,Get Your Heart Pumping with These Seasonal Fit...,"Hey there, fitness enthusiasts! Today, I'm her...","[He, there, fitnes, enthusiasts, Today, I', he...","[[0.0008336128084920347, -0.008106449618935585...","[t, seasona, i]"
4,The Power of Herbal Teas for Optimal Health,"Hey health enthusiast friends! Today, I want t...","[He, healt, enthusias, friends, Today, , wan, ...","[[-0.06643760204315186, -0.01917257159948349, ...","[tea, t, i]"
5,The Ins and Outs of Proper Handwashing,"Yo yo yo, what up my peeps? Today I wanna talk...","[Y, y, yo, wha, u, m, peeps, Toda, , wann, tal...","[[-0.048634909093379974, -0.01007450744509697,...","[clean, , an]"
